In [ ]:
import glob
import json
import re

import ipywidgets as widgets
import pandas as pd
import seaborn.objects as so
from IPython.display import display
from itables import show
from pandas import DataFrame

In [ ]:
# Load results

results_folder = "../bench/results"

# results/<tag>/<timestamp>/<size>.json
file_patter = re.compile(f"{results_folder}/([^/]+)/([^/]+)/([^/]+).json")
file_glob = f"{results_folder}/*/*/*.json"


def json_into_df(file_path: str) -> DataFrame:
    match = file_patter.fullmatch(file_path)
    if match is None:
        raise

    with open(file_path) as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    df["tag"] = match.group(1)
    df["timestamp"] = match.group(2)
    df["size"] = int(match.group(3))
    df["tag/timestamp"] = df["tag"] + "/" + df["timestamp"]
    return df


def get_all_results():
    jsons = glob.glob(file_glob)
    dfs = map(json_into_df, jsons)
    return pd.concat(dfs, ignore_index=True)


df = get_all_results()

In [ ]:
# Derive metrics


def fix(row):
    try:
        return int(row.rate.removesuffix("bps"))
    except:
        return None


df["rate_bps"] = df.apply(fix, axis=1)

df["rate_gbps"] = df["rate_bps"] / 1000**3

df["actual_duration_s"] = df["actual_duration_ms"] / 1000

df["pps_out"] = df["stats.total.opackets"] / df["actual_duration_s"]
df["mpps_out"] = df["pps_out"] / 1000000

df["pps"] = df["stats.total.ipackets"] / df["actual_duration_s"]
df["mpps"] = df["pps"] / 1000000

df["actual_rate_bps_out"] = (df["stats.total.obytes"] * 8) / df["actual_duration_s"]
df["actual_rate_gbps_out"] = df["actual_rate_bps_out"] / 1000**3

df["actual_rate_bps"] = (df["stats.total.ibytes"] * 8) / df["actual_duration_s"]
df["actual_rate_gbps"] = df["actual_rate_bps"] / 1000**3

df["gap"] = df["actual_rate_bps_out"] / df["rate_bps"] * 100

In [ ]:
sorted(list(df.columns.values))

In [ ]:
show(df)

In [ ]:
def print_parameters(df):
    ports = df["ports"].astype(str).unique()
    threshold = df["threshold"].unique()
    precision_bps = df["precision_bps"].unique()
    wait_time_s = df["wait_time_s"].unique()
    expected_duration_ms = df["expected_duration_ms"].unique()
    rx_delay_mx = df["rx_delay_ms"].unique()
    size = df["size"].unique()
    print(
        f"ports={ports} threshold={threshold} precision_bps={precision_bps} wait_time_s={wait_time_s} expected_duration_ms={expected_duration_ms} rx_delay_mx={rx_delay_mx} size={size}"
    )

In [ ]:
tag_dropdown = widgets.Dropdown(options=df["tag"].unique(), description="Tag:")
timestamp_dropdown = widgets.Dropdown(description="Timestamp:")
size_dropdown = widgets.Dropdown(description="Packet Size:")


def on_tag_change(change):
    selected_tag = change["new"]
    new_timestamps = df[df["tag"] == selected_tag]["timestamp"].unique()
    timestamp_dropdown.options = new_timestamps


def on_timestamp_change(change):
    selected_tag = tag_dropdown.value
    selected_timestamp = change["new"]

    if selected_timestamp is not None:
        new_sizes = df[
            (df["tag"] == selected_tag) & (df["timestamp"] == selected_timestamp)
        ]["size"].unique()
        size_dropdown.options = new_sizes


tag_dropdown.observe(on_tag_change, names="value")
timestamp_dropdown.observe(on_timestamp_change, names="value")

In [ ]:
def plot_drop_by_iteration(tag, timestamp, size):
    run = df[
        (df["tag"] == tag)
        & (df["timestamp"] == timestamp)
        & (df["size"] == size)
        & (df["iteration"] > 10)
    ]

    display(
        so.Plot(run, x="iteration", y="lost_percentage")
        .add(so.Line())
        .add(so.Line(color="red"), x=run["iteration"], y=run["threshold"].iloc[0])
        .label(title=f"Tag: {tag}, Timestamp: {timestamp}, Packet size: {size} bytes")
        .limit(y=(0, None))
    )

    print_parameters(run)


widgets.interactive(
    plot_drop_by_iteration, tag=tag_dropdown, timestamp=timestamp_dropdown, size=size_dropdown
)

In [ ]:
def plot_rates_by_iteration(tag, timestamp, size):
    run = df[(df["tag"] == tag) & (df["timestamp"] == timestamp) & (df["size"] == size)]
    print_parameters(run)
    run = run.melt(
        id_vars=["iteration"],
        value_vars=[
            "actual_rate_gbps",
            "actual_rate_gbps_out",
            "rate_gbps",
        ],
        var_name="in_out",
        value_name="y",
    )
    display(
        so.Plot(run, x="iteration", y="y", color="in_out")
        .add(so.Line())
        .label(title=f"Tag: {tag}, Timestamp: {timestamp}, Packet size: {size} bytes")
    )


widgets.interactive(
    plot_rates_by_iteration,
    tag=tag_dropdown,
    timestamp=timestamp_dropdown,
    size=size_dropdown,
)

In [ ]:
def plot_last_rate_by_size(tag, timestamp):
    run = df[(df["tag"] == tag) & (df["timestamp"] == timestamp)]
    display(
        so.Plot(
            run.loc[run.groupby("size")["iteration"].idxmax()],
            x="size",
            y="actual_rate_gbps",
        )
        .add(so.Line())
        .add(so.Dot())
        .label(title=f"Tag: {tag}, Timestamp: {timestamp}")
    )
    print_parameters(run)


widgets.interactive(plot_last_rate_by_size, tag=tag_dropdown, timestamp=timestamp_dropdown)

In [ ]:
def plot_latency_by_size(tag, timestamp):
    run = df[(df["tag"] == tag) & (df["timestamp"] == timestamp)]
    display(
        so.Plot(
            run.loc[run.groupby("size")["iteration"].idxmax()],
            x="size",
            y="stats.latency.0.latency.average",
            ymin="stats.latency.0.latency.total_min",
            ymax="stats.latency.0.latency.total_max",
        )
        .add(so.Line())
        .add(so.Band())
        .label(title=f"Tag: {tag}, Timestamp: {timestamp}")
    )
    print_parameters(run)


widgets.interactive(
    plot_latency_by_size, tag=tag_dropdown, timestamp=timestamp_dropdown
)

In [ ]:
def plot_latency_by_iteration(tag, timestamp, size):
    run = df[(df["tag"] == tag) & (df["timestamp"] == timestamp) & (df["size"] == size)]
    display(
        so.Plot(
            run,
            x="iteration",
            y="stats.latency.0.latency.average",
            ymin="stats.latency.0.latency.total_min",
            ymax="stats.latency.0.latency.total_max",
        )
        .add(so.Line())
        .add(so.Band())
        .label(title=f"Tag: {tag}, Timestamp: {timestamp}")
    )
    print_parameters(run)


widgets.interactive(
    plot_latency_by_iteration, tag=tag_dropdown, timestamp=timestamp_dropdown, size=size_dropdown
)